In [1]:
import time
import socket

# Create a UDP socket
clientSocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
clientSocket.settimeout(1)  # Set a timeout of 1 second

# Server details
serverAddress = ('localhost', 12000)

# Variables to track consecutive packet losses
consecutive_losses = 0
num_heartbeats = 100  # Set the number of heartbeats to send
max_consecutive_losses = 3  # Assume the server is down if we miss 3 responses

for sequence_number in range(1, num_heartbeats + 1):
    # Record the time the heartbeat was sent
    start_time = time.time()

    # Create the heartbeat message (format: "Heartbeat <sequence_number> <timestamp>")
    message = f"Heartbeat {sequence_number} {start_time}"

    try:
        # Send the heartbeat message to the server
        clientSocket.sendto(message.encode(), serverAddress)
        print(f"Sent: {message}")

        # Receive the server's response
        response, server = clientSocket.recvfrom(1024)

        # If we receive a response, reset the consecutive_losses counter
        consecutive_losses = 0

        # Print the server's response
        print(f"Reply from {server}: {response.decode()}")

    except socket.timeout:
        # Increment consecutive losses if a response is not received within 1 second
        consecutive_losses += 1
        print(f"Heartbeat {sequence_number}: Request timed out")

        # If 3 consecutive losses are detected, assume the server has stopped
        if consecutive_losses >= max_consecutive_losses:
            print("Server is assumed to have stopped after 3 consecutive timeouts.")
            break

# Close the socket after the loop finishes
clientSocket.close()

Sent: Heartbeat 1 1726493428.970259
Reply from ('127.0.0.1', 12000): Seq: 1, Time Difference: 0.003999 seconds
Sent: Heartbeat 2 1726493428.974258
Heartbeat 2: Request timed out
Sent: Heartbeat 3 1726493429.977018
Heartbeat 3: Request timed out
Sent: Heartbeat 4 1726493430.9781954
Heartbeat 4: Request timed out
Server is assumed to have stopped after 3 consecutive timeouts.
